# SDXL+ControlNetのサンプル利用コード
##利用方法
・基本的には全てのセルを実行することで利用できます。

・GPUを利用しましょう。（利用制限があるのでこまめにインスタンスは削除してください）

（変更する際は，「ランタイム」→「ランタイムのタイプを変更」からGPUを選択してください）

##注意点
・本ファイルの相対的な場所は変更しないでください。

・本ファイルの名前を変更しないでください。

変更する場合は，[2]セル目のファイル名も一緒に変更してください。

・[1]セル目の実行時にセッションの再起動を求められることがあります。

その場合はポップアップに従って再起動したあと，全てのセルを再度実行してください。

In [1]:
#Google Driveのフォルダをマウント（認証入る）
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#SDXL で必要なモジュールのインストール

!pip install -U bitsandbytes diffusers optimum-quanto peft tensorflow-metadata transformers scikit-learn ftfy accelerate invisible_watermark safetensors controlnet-aux mediapipe timm



  Using cached bitsandbytes-0.43.3-py3-none-manylinux_2_24_x86_64.whl.metadata (3.5 kB)
  Using cached diffusers-0.30.2-py3-none-any.whl.metadata (18 kB)
  Using cached optimum_quanto-0.2.4-py3-none-any.whl.metadata (13 kB)
  Using cached peft-0.12.0-py3-none-any.whl.metadata (13 kB)
  Using cached scikit_learn-1.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached accelerate-0.34.2-py3-none-any.whl.metadata (19 kB)
  Using cached invisible_watermark-0.2.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached controlnet_aux-0.0.9-py3-none-any.whl.metadata (6.5 kB)
  Using cached mediapipe-0.10.15-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.7 kB)
  Using cached timm-1.0.9-py3-none-any.whl.metadata (42 kB)
  Using cached timm-0.6.7-py3-none-any.whl.metadata (33 kB)
INFO: pip is looking at multiple versions of mediapipe to determine which version is compatible with other requirements. This could take a while.
  Using cached 

下記のコードを実行すると，認証が求められるので，認証してください。（Google colabにマイドライブの中を読めるようにする認証です）

In [3]:

# カレントディレクトリを本ファイルが存在するディレクトリに変更する。
import glob
import os
pwd = os.path.dirname(glob.glob('/content/drive/MyDrive/colabzenn/colab_SDXLControlNet2_sample/SDXLControlNet_sample.ipynb', recursive=True)[0])
print(pwd)

%cd $pwd
!pwd


/content/drive/MyDrive/colabzenn/colab_SDXLControlNet2_sample
/content/drive/MyDrive/colabzenn/colab_SDXLControlNet2_sample
/content/drive/MyDrive/colabzenn/colab_SDXLControlNet2_sample


'import sys\nsys.path.append("/content/diffusers/src")'

In [4]:
#モジュールをimportする
from module.module_sdxl import SDXL
import time

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

モデルを変更する場合は，下記のconfigを変更してください。

「;」を付けるとコメントアウトできます。


In [5]:
#モデルの設定を行う。

config_text = """
[SDXL]
device = auto
n_steps=28
seed=42

;from_single_file = True
;base_model_path = stabilityai/stable-diffusion-xl-base-1.0
base_model_path = Asahina2K/Animagine-xl-3.1-diffuser-variant-fp16
;base_model_path = ./inputs/pony/ponyDiffusionV6XL_v6StartWithThisOne.safetensors
;base_model_path = ./inputs/fudukiMix_v20.safetensors
;from_single_vae_file = True
;base_vae_model = ./inputs/pony/sdxl_vae.safetensors

use_controlnet = True

;controlnet_path0 = ./inputs/CN-anytest_v3-45000/
;from_single_controlnet_file0 = False
;use_controlnet_variant0 = True
controlnet_path0 = ./inputs/CN-anytest_v4/
from_single_controlnet_file0 = False
use_controlnet_variant0 = False
;controlnet_path1 = diffusers/controlnet-depth-sdxl-1.0
;from_single_controlnet_file1 = False
;use_controlnet_variant1 = True
;controlnet_path2 = xinsir/controlnet-openpose-sdxl-1.0
;from_single_controlnet_file2 = False
;use_controlnet_variant2 = True
;controlnet_path3 = diffusers/controlnet-canny-sdxl-1.0
;from_single_controlnet_file3 = False
;use_controlnet_variant3 = True
;controlnet_path4 = diffusers/controlnet-zoe-depth-sdxl-1.0
;from_single_controlnet_file4 = False
;use_controlnet_variant4 = True

;残念ながら下記は動かない
;controlnet_path0 = https://huggingface.co/2vXpSwA7/iroiro-lora/blob/main/test_controlnet2/CN-anytest_v4-marged.safetensors
;controlnet_config_repo0 = diffusers/controlnet-canny-sdxl-1.0
;from_single_controlnet_file0 = True
;use_controlnet_variant0 = False

;modeと同じ名前の参照画像が格納されているとする。（tile.pngなど）
control_mode0 = tile
;control_mode1 = depth
;control_mode2 = openpose_full
;control_mode3 = canny
;control_mode4 = zoe_depth

;以下は待機用
;control_mode0 = blur
;control_mode0 = gray
;control_mode0 = lq
;control_mode0 = openpose_face
;control_mode0 = openpose_faceonly
;control_mode0 = openpose



;lora_weight_repo0 = erohinem/LoRA
;lora_weight_path0 = Dreamyvibes-lora.safetensors
;lora_weight_repo1 = xxx
;lora_weight_path1 = xxx.safetensors
;lora_scale0 = 1.0
;lora_scale1 = 1.0
;trigger_word0 = "Dreamyvibes Artstyle"
;trigger_word1 = "xxx"

;select_solver = LCM
select_solver = DPM
;select_solver = Eulera
;select_solver = FMEuler

use_karras_sigmas = True
scheduler_algorithm_type = dpmsolver++
solver_order = 2

cfg_scale = 7.0
width = 832
height = 1216
output_type = pil
aesthetic_score = 6
negative_aesthetic_score = 2.5

save_latent_simple = False
save_latent_overstep = False
save_latent_approximation = False
save_predict_skip_x0 = False


"""

with open("configs/config.ini", "w", encoding="utf-8") as f:
  f.write(config_text)

生成する画像を決定するプロンプトを指定します。

negative_promptを設定することで，ここに書かれている内容を回避できる。


In [6]:
#読み上げるプロンプトを設定する。

# not use controlnet
#main_prompt = """
#modern anime style, a cute Japanese idle singer girl, long Yellowish-white hair with red small ribbon,red eyes,small red hat ,She is singing ,holding a microphone, wearing white idle costume, standing on the stage ,This is a flashy concert with 3D holograms and laser effects
#"""

#use controlnet depth
#main_prompt = """
#modern anime style, a cute Japanese idle singer girl, long Yellowish-white hair with red small ribbon,red eyes,small red hat ,She sings with a smile on her face , wearing white idle costume, standing on the stage ,This is a flashy concert with 3D holograms and laser effects
#"""

#double lora and anime lora only
#main_prompt = """
#a cute Japanese idle singer girl, long Yellowish-white hair with red small ribbon,red eyes,small red hat ,She sings with a smile on her face , wearing white idle costume, standing on the stage
#"""


#use controlnet canny
#main_prompt = """
#modern anime style, a cute Japanese idle singer girl, long Yellowish-white hair with red small ribbon,red eyes
#"""

#figre black
#main_prompt = """
#a photo of an anime character figure,Navy blue uniform, gray skirt, black tights, black hair, yellow-green tie, black shoes
#"""

#figre red
#main_prompt = """
#a photo of an anime character figure,Yellowish-white hair , red ribbon,red eyes,white hat, white costume, black microphone, bare legs,Skin color is white
#"""

#scarf scketch
main_prompt = """
scarf, Yellowish-white hair , red ribbon,red eyes,white hat, white costume, black microphone, bare legs
"""

#main_prompt = """
#Convert the presented cartoon image to a realistic figure image of the same character.
#"""

neg_prompt="""
nsfw, lowres, (bad), text, error, fewer, extra, missing, worst quality, jpeg artifacts, low quality, watermark, unfinished, displeasing, oldest, early, chromatic aberration, signature, extra digits, artistic error, username, scan, [abstract]
"""

#コントロールネットに入力する参照画像の名前はcontrolnetのモードと一致させてください。（tile.png, depth.jpegなど）
input_refer_image_folder = "./inputs/refer/sketch2"
#コントロールネットに入力する変換後の参照画像の格納フォルダ。命名規則は上記と同様
output_refer_image_folder = "./inputs/refer_prepared/sketch2"





In [7]:
sdxl = SDXL()
#指定しているControlNetで利用できる"./inputs/refer_prepared/tile.png"などがすでにある場合は、下記はコメントアウトしても良い
sdxl.prepare_multi_referimage(input_refer_image_folder = input_refer_image_folder,output_refer_image_folder = output_refer_image_folder, low_threshold = 100, high_threshold = 200, noise_level=25, blur_radius=5)



loaded controlnet


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

text_encoder_2/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

model.fp16.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/5.14G [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

loaded base model
image_paths=['./inputs/refer/sketch2/tile.png']
self.control_modes_list=['tile', None, None, None, None]
image_paths_control=['./inputs/refer/sketch2/tile.png'], output_image_paths=['./inputs/refer_prepared/sketch2/tile.png']


In [8]:
for i in range(5):
      start = time.time()
      image = sdxl.generate_image(main_prompt,neg_prompt = neg_prompt, image_path = output_refer_image_folder, controlnet_conditioning_scale = [0.8])
      print("generate image time: ", time.time()-start)
      image.save("./outputs/SDXL_result_{}.png".format(i))

use controlnet mode:  ['tile', None, None, None, None]
Image loaded from file path.


  0%|          | 0/28 [00:00<?, ?it/s]

generate image time:  34.6427788734436
use controlnet mode:  ['tile', None, None, None, None]
Image loaded from file path.


  0%|          | 0/28 [00:00<?, ?it/s]

KeyboardInterrupt: 